<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />


<br><h1 style="list-style:none"> A3 - Analysis Report</h1><br>
<b>DAT-5303 | Machine Learning</b><br>
Fabian Jaskotka<br>
Hult International Business School<br><br>

<p>Apprentice Chef, Inc.<br>
   Key Insights into:
</p>
    <ul>
        <li>Which factors drive Revenue</li>
        <li>Which customers are most likely to subscribe to the halfway there service</li>
    </ul>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />


<br>

# Management Report

<b>Introduction</b><br><br>
Following the request of Apprentice Chef to help to better understand how much revenue to expect from their customers as well as predict which customers would sign up for the new <i>Halfway There</i> program, machine learning models have been developed and their derived insights are presented in this report.<br><br>

<b>Expecting customer revenue</b><br><br>
The most relevant insight for the expectation of revenue of a customer appears to be the Average Preparation Instruction video watching time. A 1% increase in watching times can be associated with a 50% increase in expected revenue. Therefore, instruction videos seem very helpful and shall be advertised more to clients.<br>
Also the rating of the meals affects the revenue that can be expected from a customer. A customer with a 4-star median meal rating is expected to generate a revenue that is about 12% higher than that of a customer with a 1-star median meal rating. Hence, customer satisfaction shall be focused on and concrete improvement steps analyzed.<br>
(ARD Model, R-Square: 0.792)<br><br>

<b>Predicting Campaign Acceptance</b><br><br>
The most significant impact on the acceptance of the campaign comes from the email addresses. Especially people that signed up with junk classified email addresses (@me.com, etc.) are particularly likely to accept the offer. Therefore, focus shall be put on the advertisement to people with these email addresses.<br>
Additionally, the number of cancellations before noon plays an important role in predicting the customer’s acceptance of the <i>Halfway There</i> campaign. Customers with less than 2 cancellations are more likely to accept. Marketing the campaign to these customers can be considered especially promising.<br>
(Classification Tree, AUC-score: 0.73)<br><br>

<b>Conclusion</b><br><br>
The four insights presented above paired with their brief recommendations shall enable Apprentice Chef to take control over customer generated revenue and allow for an efficient planing for a successful <i>Halfway There</i> campaign.

# Loading Data

In [ ]:
#importing necessary packages
import random as rand
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gender_guesser.detector as gender
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf

#for linear regression
import sklearn.linear_model 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

#for classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix         # confusion matrix
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier      # classification trees
from sklearn.tree import export_graphviz             # exports graphics
from six import StringIO                             # saves objects in memory
from IPython.display import Image                    # displays on frontend
import pydotplus                                     # interprets dot objects
from sklearn.model_selection import RandomizedSearchCV     # hyperparameter tuning
from sklearn.metrics import make_scorer              # customizable scorer
from sklearn.ensemble import RandomForestClassifier     # random forest
from sklearn.ensemble import GradientBoostingClassifier # gbm

In [ ]:
#setting seed
rand.seed(a=327)

#defining filepath for dataset
file = "./datasets/Apprentice_Chef_Dataset.xlsx"

#matching datatypes
data_types = {"AVG_CLICKS_PER_VISIT" : float}

#importing dataset
chef = pd.read_excel(io = file,
                    dtype = data_types)

#changing column name of Largest_Order_Size
chef.columns = ['REVENUE', 'CROSS_SELL_SUCCESS', 'NAME', 'EMAIL', 'FIRST_NAME',
       'FAMILY_NAME', 'TOTAL_MEALS_ORDERED', 'UNIQUE_MEALS_PURCH',
       'CONTACTS_W_CUSTOMER_SERVICE', 'PRODUCT_CATEGORIES_VIEWED',
       'AVG_TIME_PER_SITE_VISIT', 'MOBILE_NUMBER', 'CANCELLATIONS_BEFORE_NOON',
       'CANCELLATIONS_AFTER_NOON', 'TASTES_AND_PREFERENCES', 'PC_LOGINS',
       'MOBILE_LOGINS', 'WEEKLY_PLAN', 'EARLY_DELIVERIES', 'LATE_DELIVERIES',
       'PACKAGE_LOCKER', 'REFRIGERATED_LOCKER', 'AVG_PREP_VID_TIME',
       'AVG_NUMBER_MEALS', 'MASTER_CLASSES_ATTENDED', 'MEDIAN_MEAL_RATING',
       'AVG_CLICKS_PER_VISIT', 'TOTAL_PHOTOS_VIEWED']


# formatting and printing the dimensions of the dataset
print(f"""
Size of Original Dataset
------------------------
Observations: {chef.shape[0]}
Features:     {chef.shape[1]}
""")

# Data Preparation

## Anomaly Handling

In [ ]:
#checking for missing values
chef.isnull().any().any()

In [ ]:
#dropping Family name variable
chef = chef.drop('FAMILY_NAME', axis = 1)

## User-defined Functions

In [ ]:
########################################
# optimal_neighbors
######################################## 
def optimal_neighbors(X_data,
                      y_data,
                      standardize = True,
                      pct_test=0.25,
                      seed=219,
                      response_type='reg',
                      max_neighbors=20,
                      show_viz=True):
    """
Exhaustively compute training and testing results for KNN across
[1, max_neighbors]. Outputs the maximum test score and (by default) a
visualization of the results.
PARAMETERS
----------
X_data        : explanatory variable data
y_data        : response variable
standardize   : whether or not to standardize the X data, default True
pct_test      : test size for training and validation from (0,1), default 0.25
seed          : random seed to be used in algorithm, default 219
response_type : type of neighbors algorithm to use, default 'reg'
    Use 'reg' for regression (KNeighborsRegressor)
    Use 'class' for classification (KNeighborsClassifier)
max_neighbors : maximum number of neighbors in exhaustive search, default 20
show_viz      : display or surpress k-neigbors visualization, default True
"""    
    
    
    if standardize == True:
        # optionally standardizing X_data
        scaler             = StandardScaler()
        scaler.fit(X_data)
        X_scaled           = scaler.transform(X_data)
        X_scaled_df        = pd.DataFrame(X_scaled)
        X_data             = X_scaled_df



    # train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_data,
                                                        y_data,
                                                        test_size = pct_test,
                                                        random_state = seed)


    # creating lists for training set accuracy and test set accuracy
    training_accuracy = []
    test_accuracy = []
    
    
    # setting neighbor range
    neighbors_settings = range(1, max_neighbors + 1)


    for n_neighbors in neighbors_settings:
        # building the model based on response variable type
        if response_type == 'reg':
            clf = KNeighborsRegressor(n_neighbors = n_neighbors)
            clf.fit(X_train, y_train)
            
        elif response_type == 'class':
            clf = KNeighborsClassifier(n_neighbors = n_neighbors)
            clf.fit(X_train, y_train)            
            
        else:
            print("Error: response_type must be 'reg' or 'class'")
        
        
        # recording the training set accuracy
        training_accuracy.append(clf.score(X_train, y_train))
    
        # recording the generalization accuracy
        test_accuracy.append(clf.score(X_test, y_test))


    # optionally displaying visualization
    if show_viz == True:
        # plotting the visualization
        fig, ax = plt.subplots(figsize=(12,8))
        plt.plot(neighbors_settings, training_accuracy, label = "training accuracy")
        plt.plot(neighbors_settings, test_accuracy, label = "test accuracy")
        plt.ylabel("Accuracy")
        plt.xlabel("n_neighbors")
        plt.legend()
        plt.show()
    
    
    # returning optimal number of neighbors
    print(f"The optimal number of neighbors is: {test_accuracy.index(max(test_accuracy))+1}")
    return test_accuracy.index(max(test_accuracy))+1


########################################
# visual_cm
########################################
def visual_cm(true_y, pred_y, labels = None):
    """
Creates a visualization of a confusion matrix.

PARAMETERS
----------
true_y : true values for the response variable
pred_y : predicted values for the response variable
labels : , default None
    """
    # visualizing the confusion matrix

    # setting labels
    lbls = labels
    

    # declaring a confusion matrix object
    cm = confusion_matrix(y_true = true_y,
                          y_pred = pred_y)


    # heatmap
    sns.heatmap(cm,
                annot       = True,
                xticklabels = lbls,
                yticklabels = lbls,
                cmap        = 'Blues',
                fmt         = 'g')


    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix of the Classifier')
    plt.show()

# Feature Engineering

## Revenue Transformation

In [ ]:
#Logarithmic transformation of Revenue
chef['LOG_REVENUE'] = np.log10(chef['REVENUE'])

## Photos viewed at all

In [ ]:
#dummy variables
chef['PHOTOS_VIEWED_AT_ALL'] = 0


#creating loop to iterate over dataset
for index, value in chef.iterrows():
        
    #assigning 1 to customers that already viewed photos
    if chef.loc[index, 'TOTAL_PHOTOS_VIEWED'] > 0:
        chef.loc[index, 'PHOTOS_VIEWED_AT_ALL'] = 1
 

## Cancellations after noon per Meal ordered

In [ ]:
#cancellation after noon per meal ordered
chef['CANCELLATION_AFTER_NOON_PMO'] = chef['CANCELLATIONS_AFTER_NOON'] / chef['TOTAL_MEALS_ORDERED']

## Contacts with customer service

### Per Meal Ordered

In [ ]:
#generating feature as ratio of service requests per meal ordered
chef['CONTACTS_W_CUSTOMER_SERVICE_PMO'] =  chef['TOTAL_MEALS_ORDERED'] / chef['CONTACTS_W_CUSTOMER_SERVICE']

### Logarithmic Transformation

In [ ]:
#transforming variable
chef['LOG_CONTACTS_W_CUSTOMER_SERVICE_PMO'] = np.log10(chef['CONTACTS_W_CUSTOMER_SERVICE_PMO'])

### Split at 0.25

In [ ]:
#placeholder list
placeholder_lst = []
#dummy variables for CONTACTS_W_CUSTOMER_SERVICE_PMO
for item in chef['LOG_CONTACTS_W_CUSTOMER_SERVICE_PMO']:
    if item > 0.25:
        placeholder_lst.append(1)
        
    else:
        placeholder_lst.append(0)

#adding new feature to dataset
chef['CS_PMO_MIN_2'] = pd.Series(placeholder_lst)

### Split at 10 Contacts

In [ ]:
#creating placeholder list
placeholder_lst = []

#for loop to loop over Contacts_W_Customer service
for item in chef['CONTACTS_W_CUSTOMER_SERVICE']:
    
    if item <= 10:
        placeholder_lst.append(0)
        
    else:
        placeholder_lst.append(1)
        
chef['CONTACTS_W_CUSTOMER_SERVICE_ABOVETEN'] = pd.Series(placeholder_lst)

## Total Meals Ordered Split

In [ ]:
#creating feature for above 20 meals ordered
placeholder_lst = []

for item in chef['TOTAL_MEALS_ORDERED']:
    if item > 20:
        placeholder_lst.append(1)
    else:
        placeholder_lst.append(0)

#adding new feature to dataset
chef['TOTAL_MEALS_ABOVE20'] = pd.Series(placeholder_lst)

## Unique Meals per Meals Ordered

In [ ]:
#creating feature
chef['UNIQUE_MEALS_PURCH_PMO'] = chef['UNIQUE_MEALS_PURCH'] / chef['TOTAL_MEALS_ORDERED']

## Masterclasses attended

In [ ]:
#creating dummy variable
master_dummy = pd.get_dummies(chef['MASTER_CLASSES_ATTENDED'])

#renaming the columns of the dummy dataset
master_dummy.columns = ['NO_MASTER_CLASSES_ATTENDED', 'ONE_MASTER_CLASSES_ATTENDED', 
                        'TWO_MASTER_CLASSES_ATTENDED', 'THREE_MASTER_CLASSES_ATTENDED']

#adding dummy variables as new features
chef = chef.join([master_dummy.iloc[:, 1:3]])

## Median Meal Rating

In [ ]:
#creating dummy variable
rating_dummy = pd.get_dummies(chef['MEDIAN_MEAL_RATING'])

#renaming columns of dummy dataset
rating_dummy.columns = ['ONE_MEDIAN_MEAL_RATING', 'TWO_MEDIAN_MEAL_RATING', 
                       'THREE_MEDIAN_MEAL_RATING', 'FOUR_MEDIAN_MEAL_RATING',
                       'FIVE_MEDIAN_MEAL_RATING']

#adding new dummy variables to chef dataset
chef = chef.join([rating_dummy.iloc[:, 1:4]])

## Average Clicks per Site Visit Split

In [ ]:
#creating placeholder list
placeholder_lst = []

#looping over Average click per visit feature
for item in chef['AVG_CLICKS_PER_VISIT']:
    if item > 10:
        placeholder_lst.append(1)
        
    else:
        placeholder_lst.append(0)

#transforming list to Series and adding to dataset
chef['AVG_CLICKS_PER_VISIT_ABOVETEN'] = pd.Series(placeholder_lst)

## Number of Names

In [ ]:
#########################
# text_split_feature
#########################
def text_split_feature(col, df, sep=' ', new_col_name='number_of_names'):
    """
Splits values in a string Series (as part of a DataFrame) and sums the number
of resulting items. Automatically appends summed column to original DataFrame.

PARAMETERS
----------
col          : column to split
df           : DataFrame where column is located
sep          : string sequence to split by, default ' '
new_col_name : name of new column after summing split, default
               'number_of_names'
"""
    
    df[new_col_name] = 0
    
    
    for index, val in df.iterrows():
        df.loc[index, new_col_name] = len(df.loc[index, col].split(sep = ' '))

In [ ]:
#calling text_split_function to count number of names
text_split_feature(col = 'NAME', 
                   df = chef,
                  new_col_name = 'NUMBER_OF_NAMES')

#dropping the name feature
chef = chef.drop('NAME', axis =1)

## Gender

In [ ]:
#loading in the gender List
gender_file = "./datasets/chef_gender.xlsx"

gender = pd.read_excel(io = gender_file)

In [ ]:
#creating dummy variables
one_hot_gender = pd.get_dummies(gender)

one_hot_gender.columns = ['andy', 'FEMALE', 'MALE', 'mostly_female', 
                          'mostly_male','unknown']

#dropping variables related to gender
chef = chef.drop(['FIRST_NAME'], axis = 1)

#joining dummy variables to chef
chef = chef.join(one_hot_gender.iloc[:, 1:3])

## Email

In [ ]:
#generating empty list
placeholder_lst = []

for index, value in chef.iterrows():
    
    split_email = chef.loc[index, 'EMAIL'].split(sep = '@')
    
    placeholder_lst.append(split_email)
    
email_df = pd.DataFrame(placeholder_lst)

email_df.columns = ['0', 'EMAIL_DOMAIN']

#adding email domain feature to chef dataset
chef = pd.concat([chef, email_df['EMAIL_DOMAIN']], axis = 1)

In [ ]:
#grouping mail domains in professional, personal and junk
professional_mail = ['@mmm.com', '@amex.com', '@apple.com', '@boeing.com', '@caterpillar.com', 
                    '@chevron.com', '@cisco.com', '@cocacola.com', '@disney.com', '@dupont.com', 
                    '@exxon.com', '@ge.org', '@goldmansacs.com', '@homedepot.com', '@ibm.com', 
                    '@intel.com', '@jnj.com', '@jpmorgan.com', '@mcdonalds.com', '@merck.com', 
                     '@microsoft.com', '@nike.com', '@pfizer.com', '@pg.com', '@travelers.com',
                    '@unitedtech.com', '@unitedhealth.com', '@verizon.com', '@visa.com', '@walmart.com']

personal_mail = ['@gmail.com', '@yahoo.com', '@protonmail.com']

junk_mail = ['@me.com', '@aol.com', '@hotmail.com', '@live.com', '@msn.com', '@passport.com']


#placeholder list
placeholder_lst = []


#loop to go over domains and assign categories
for domain in chef['EMAIL_DOMAIN']:
    
    if '@' + domain in professional_mail:
        placeholder_lst.append('PROFESSIONAL_MAIL')
        
    elif '@' + domain in personal_mail:
        placeholder_lst.append('PERSONAL_MAIL')
        
    elif '@' + domain in junk_mail:
        placeholder_lst.append('JUNK_MAIL')

#creating new feature, gomain group
chef['DOMAIN_GROUP'] = pd.Series(placeholder_lst)

In [ ]:
#one-hot encoding of domain group variable
one_hot_domain = pd.get_dummies(chef['DOMAIN_GROUP'])

#dropping categorical variables after encoding
chef = chef.drop('DOMAIN_GROUP', axis = 1 )

#adding dummy variables to dataset
chef = chef.join([one_hot_domain])

In [ ]:
#dropping the variables of types object
chef = chef.drop(['EMAIL', 'EMAIL_DOMAIN'], axis = 1)

## Logarithmic Transformation

In [ ]:
#dictionary with all features to be transformed
log_features = ['TOTAL_MEALS_ORDERED', 'CONTACTS_W_CUSTOMER_SERVICE', 
                'AVG_PREP_VID_TIME', 'AVG_NUMBER_MEALS',
               'CONTACTS_W_CUSTOMER_SERVICE_PMO','UNIQUE_MEALS_PURCH_PMO']

#loop for transformation of all features from dictionary
for feature in log_features:
    
    #logrithmic transformation of features
    chef['LOG_'+feature] = np.log10(chef[feature])
    

#dropping normal form of transformed variables
chef = chef.drop(log_features, axis = 1)

# Developing Models

## Train-Test Split

In [ ]:
#dictionaryfor explainingvariables

candidate_dict = {
    'regression':['CROSS_SELL_SUCCESS','UNIQUE_MEALS_PURCH',
                  'REFRIGERATED_LOCKER','MEDIAN_MEAL_RATING',
                  'PHOTOS_VIEWED_AT_ALL','CANCELLATION_AFTER_NOON_PMO',
                  'LOG_CONTACTS_W_CUSTOMER_SERVICE',
                  'CONTACTS_W_CUSTOMER_SERVICE_ABOVETEN',
                  'LOG_CONTACTS_W_CUSTOMER_SERVICE_PMO','CS_PMO_MIN_2',
                  'TOTAL_MEALS_ABOVE20','LOG_TOTAL_MEALS_ORDERED',
                  'LOG_UNIQUE_MEALS_PURCH_PMO','LOG_AVG_PREP_VID_TIME',
                  'LOG_AVG_NUMBER_MEALS','ONE_MASTER_CLASSES_ATTENDED',
                  'TWO_MASTER_CLASSES_ATTENDED','THREE_MEDIAN_MEAL_RATING',
                  'FOUR_MEDIAN_MEAL_RATING','AVG_CLICKS_PER_VISIT_ABOVETEN'],
    
    'classification': ['MOBILE_NUMBER','CANCELLATIONS_BEFORE_NOON',
                   'TASTES_AND_PREFERENCES','PC_LOGINS','EARLY_DELIVERIES',
                   'REFRIGERATED_LOCKER','JUNK_MAIL','PROFESSIONAL_MAIL',
                   'MALE', 'NUMBER_OF_NAMES', 'LOG_AVG_PREP_VID_TIME']
}

In [ ]:
#seperating datasets into explanatory and target for regression
chef_reg = chef.loc[ : , candidate_dict['regression']]
chef_target = chef['REVENUE']
chef_target_log = chef['LOG_REVENUE']

#seperating datasets into explanatory and target for classification
chef_clas = chef.loc[ : , candidate_dict['classification']]
chef_goal = chef.loc[ :, 'CROSS_SELL_SUCCESS']


#splitting into x_train, x_test, y_train, y_test with all variables
x_train_reg, x_test_reg, y_train_reg, y_test_reg = train_test_split(
    chef_reg, 
    chef_target_log,
    test_size = 0.25,
    random_state = 219)


#splitting into x_train, x_test, y_train, y_test with all variables
x_train_clas, x_test_clas, y_train_clas, y_test_clas = train_test_split(
    chef_clas, 
    chef_goal,
    test_size = 0.25,
    random_state = 219,
    stratify = chef_goal)

## ARD Model for Regression

In [ ]:
# INSTANTIATING a model object
ard_model = sklearn.linear_model.ARDRegression(normalize = False)


# FITTING the training data
ard_fit = ard_model.fit(x_train_reg, y_train_reg)


# PREDICTING on new data
ard_pred = ard_fit.predict(x_test_reg)


# saving scoring data for future use
model_type_3 = "ARD Regression"
ard_train_score = ard_model.score(x_train_reg, y_train_reg).round(4)
ard_test_score  = ard_model.score(x_test_reg, y_test_reg).round(4)
ard_test_gap = abs(ard_train_score - ard_test_score).round(4)


# zipping each feature name to its coefficient
ard_model_values = zip(chef_reg.columns, ard_fit.coef_.round(decimals = 5))


# setting up a placeholder list to store model features
ard_model_lst = [('intercept', ard_fit.intercept_.round(decimals = 2))]


# printing out each feature-coefficient pair one by one
for val in ard_model_values:
    ard_model_lst.append(val)
    

#setting variable
x = 6

#loop to delete coefficients
while x > 0:
    
    for feature, coefficient in ard_model_lst:
        
        if coefficient == 0:
            ard_model_lst.remove((feature, coefficient))
    
    x -= 1
  
    
#calculating model size    
model_size_ard = len(ard_model_lst)

 #printing model output
print(f"""
These are the results of the {model_type_3} model
---------------------------------------------------
Training Score : {ard_train_score}
Testing Score  : {ard_test_score}
Train-Test Gap : {ard_test_gap}
Model Size     : {model_size_ard}

""")   
    


# checking the results
for pair in ard_model_lst:
    print(pair)
    
ard_model_lst = pd.DataFrame(ard_model_lst)

ard_model_lst.columns = ['Feature', 'Coefficient']
print(ard_model_lst.sort_values('Coefficient', ascending = False))
    

# creating a dictionary for model results
model_performance_ard = {
    
    'Model Type'    : [model_type_3],
    
    
    'Training' : [ard_train_score],
                  
           
    'Testing'  : [ard_test_score],
    
    
    'Train-Test Gap' : [ard_test_gap],
                   
    
    'Model Size' : [model_size_ard],
                    
    'Model'      : [ard_model_lst],
    
    'Final'      : ["Yes"]}

## Tuned Tree for Classification

In [ ]:
########################################
# display_tree
########################################
def display_tree(tree, feature_df, height = 500, width = 800):
    """
    PARAMETERS
    ----------
    tree       : fitted tree model object
        fitted CART model to visualized
    feature_df : DataFrame
        DataFrame of explanatory features (used to generate labels)
    height     : int, default 500
        height in pixels to which to constrain image in html
    width      : int, default 800
        width in pixels to which to constrain image in html
    """

    # visualizing the tree
    dot_data = StringIO()

    
    # exporting tree to graphviz
    export_graphviz(decision_tree      = tree,
                    out_file           = dot_data,
                    filled             = True,
                    rounded            = True,
                    special_characters = True,
                    feature_names      = feature_df.columns)


    # declaring a graph object
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())


    # creating image
    img = Image(graph.create_png(),
                height = height,
                width  = width)
    
    return img

########################################
# plot_feature_importances
########################################
def plot_feature_importances(model, train, export = False):
    """
    Plots the importance of features from a CART model.
    
    PARAMETERS
    ----------
    model  : CART model
    train  : explanatory variable training data
    export : whether or not to export as a .png image, default False
    """
    
    # declaring the number
    n_features = x_train_clas.shape[1]
    
    # setting plot window
    fig, ax = plt.subplots(figsize=(12,9))
    
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(pd.np.arange(n_features), train.columns)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    
    if export == True:
        plt.savefig('Tree_Leaf_50_Feature_Importance.png')

In [ ]:
# building a model based on hyperparameter tuning results

# INSTANTIATING a logistic regression model with tuned values
tree_tuned = DecisionTreeClassifier(max_depth=3, min_samples_leaf=21, min_samples_split=4,
                       random_state=219)

# FIT step is not needed
tree_tuned_fit = tree_tuned.fit(x_train_clas, y_train_clas)

# PREDICTING based on the testing set
tree_tuned_pred = tree_tuned_fit.predict(x_test_clas)


# SCORING the results
print('Training ACCURACY:', tree_tuned.score(x_train_clas, y_train_clas).round(4))
print('Testing  ACCURACY:', tree_tuned.score(x_test_clas, y_test_clas).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test_clas,
                                          y_score = tree_tuned_pred).round(4))


# saving scoring data for future use
tree_tuned_train_score = tree_tuned.score(x_train_clas, y_train_clas).round(4) # accuracy
tree_tuned_test_score  = tree_tuned.score(x_test_clas, y_test_clas).round(4)   # accuracy


# saving the AUC score
tree_tuned_auc         = roc_auc_score(y_true  = y_test_clas,
                                     y_score = tree_tuned_pred).round(4) # auc

In [ ]:
# unpacking the confusion matrix
tuned_tree_tn, \
tuned_tree_fp, \
tuned_tree_fn, \
tuned_tree_tp = confusion_matrix(y_true = y_test_clas, y_pred = tree_tuned_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {tuned_tree_tn}
False Positives: {tuned_tree_fp}
False Negatives: {tuned_tree_fn}
True Positives : {tuned_tree_tp}
""")

In [ ]:
# calling display_tree
display_tree(tree       = tree_tuned_fit,
             feature_df = x_train_clas)

In [ ]:
# plotting feature importance
plot_feature_importances(tree_tuned_fit,
                         train  = x_train_clas,
                         export = False)